# ¿Cuál es la mejor tarifa?

Trabajas como analista para el operador de telecomunicaciones Megaline. La empresa ofrece a sus clientes dos tarifas de prepago, Surf y Ultimate. El departamento comercial quiere saber cuál de las tarifas genera más ingresos para poder ajustar el presupuesto de publicidad.

Vas a realizar un análisis preliminar de las tarifas basado en una selección de clientes relativamente pequeña. Tendrás los datos de 500 clientes de Megaline: quiénes son los clientes, de dónde son, qué tarifa usan, así como la cantidad de llamadas que hicieron y los mensajes de texto que enviaron en 2018. Tu trabajo es analizar el comportamiento de los clientes y determinar qué tarifa de prepago genera más ingresos.

**Propósito del proyecto: Determinar qué plan (surf o ultimate) es mejor para aconsejar a el departamento comercial acerca del presupuesto de publicidad.**

Se estudiará la muestra de 500 usuarios para analizar el comportamiento del consumo y se buscaran respuesta a preguntas como:

- ¿Qué plan usa más minutos, mensajes e Internet en promedio?
- ¿Tienen estos planes tendencias diferentes o similares, los usuarios exceden los servicios ya incluidos en el plan?
- ¿Qué plan genera más ingresos?

**Interpretar los datos**: contamos con 5 tablas que nos brindan información importante acerca de:

- `users` nos indica **características del usuario** como (id, nombre, edad y ciudad) lo cual nos permite hacer análisis demográficos. También nos indica datos de su (plan) y de fechas como (registro y cancelación)


- `calls` permitirá analizar el **número de llamadas y minutos totales por cada usuario** y por fecha, dado que en `users` tenemos el ID de cada usuario y su plan podemos analizar este servicio.


- `messages` permite conocer el **número total de mensajes enviados por cada usuario** y la fechas en que se realizó, nos ayudara a conocer el comportamiento por usuario y por ende por plan.


- `internet` permite conocer la **cantidad de mb usados por cada usuario en cada sesión**, esta nos permitirá conocer el uso de internet.


- `plans` contiene **la información de las tarifas de cada plan** (tanto base como adicionales) para cada uno de los servicios, será importante para determinar el ingreso por usuario y por plan teniendo en cuenta todo el uso de los servicios.

Dadas las tablas que tenemos, es importante consolidar la información por usuario en una tabla que permita determinar el uso de servicios y el ingreso originado en cada uno de ellos. Posterior mente podemos analizar para cada plan cual es el ingreso promedio.



## Inicialización

In [1]:
import json
from IPython.display import display

import pandas as pd
import numpy as np
from pandas_dq import dq_report
from cryptography.fernet import Fernet

In [2]:
# append the path of the module to the sys.path
import sys
sys.path.append('..')

from megaline import data_processing as mdp

## Cargar datos

In [3]:
# load the URLs for the data from the data_url.json file
with open('../data/data_url.json', 'r') as f:
    data_url = json.load(f)

### Users: Data Quality Check

1. `churn_date` contine valores asuenes por los usuarios activos
2. `city` contiene valores repetitivos que podemos remover como "MSA"

In [4]:
url = data_url["users"]
_ = pd.read_csv(url, nrows=2)
display(_)
_.info()

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaN
1,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   user_id     2 non-null      int64  
 1   first_name  2 non-null      object 
 2   last_name   2 non-null      object 
 3   age         2 non-null      int64  
 4   city        2 non-null      object 
 5   reg_date    2 non-null      object 
 6   plan        2 non-null      object 
 7   churn_date  0 non-null      float64
dtypes: float64(1), int64(2), object(5)
memory usage: 256.0+ bytes


In [5]:
df_users = mdp.downcast_dtypes(pd.read_csv(url,
                                    #    index_col='user_id',
                                       parse_dates=['reg_date', 'churn_date']))

mdp.check_data_quality(df_users)

### Data Quality Report

#### Missing Values

<span style="color:orange">Missing values found:</span>

churn_date    466
dtype: int64

#### Duplicate Rows

...

<span style="color:green">No highly correlated columns found</span>

#### Data Quality Report

This is a summary report. Change verbose to 1 to see more details on each DQ issue.
    All variables classified into correct types.


,DQ Issue
The Good News,"There are no duplicate rows in this dataset, There are no duplicate columns in this datatset, There are no columns with infinite values in this dataset , There are no columns with mixed (more than one) dataypes in this dataset, There are no numeric columns with outliers in this dataset, There are no highly correlated columns in the dataset., There is no target given. Hence no target leakage columns detected in the dataset"
The Bad News,"There are ID columns in the dataset. Remove them before modeling using Fix_DQ., These are zero-variance or low information columns in the dataset. Remove them before modeling., There are 1 date-time vars in the dataset. Make sure you transform them before modeling., There are 1 columns with high cardinality (>30 categories) in the dataset. Reduce them using encoding techniques or feature selection methods."


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
user_id,int16,0.0,100,1000.0,1499.0,Possible ID column: drop before modeling step.
first_name,object,0.0,91,,,458 rare categories: Too many to list. Group them into a single category or drop the categories.
last_name,object,0.0,79,,,Possible high cardinality column with 399 unique values: Use hash encoding or text embedding to reduce dimension.
age,int8,0.0,11,18.0,75.0,No issue
city,object,0.0,14,,,38 rare categories: Too many to list. Group them into a single category or drop the categories.
reg_date,datetime64[ns],0.0,53,,,Possible date-time colum: transform before modeling step.
plan,object,0.0,0,,,No issue
churn_date,datetime64[ns],93.2,5,,,"Possible Zero-variance or low information colum: drop before modeling step., 466 missing values. Impute them with mean, median, mode, or a constant value such as 123."


In [6]:
df_users = df_users.set_index("user_id")

# Separate the PII data, then standarize to capitalize the first letter of the first and last name
pii_data = df_users[['first_name', 'last_name']]
pii_data = pii_data.apply(lambda x: x.str.capitalize())

# Drop PII column
df_users.drop(['first_name', "last_name"], axis=1, inplace=True)

In [7]:
key = Fernet.generate_key()
cipher_suite = Fernet(key)

def encrypt_name(name):
    return cipher_suite.encrypt(name.encode()).decode()

# Store the pii user data
pii_data.map(encrypt_name).to_parquet("../data/clean/pii_data.parquet", compression='brotli')

# store the key
with open("../data/clean/pii_data-key.txt", "w") as f:
    f.write(key.decode())

In [8]:
if df_users["city"].str.endswith(" MSA").mean() == 1:
    df_users["city"] = df_users["city"].str[:-4]
    print("Removed MSA from city names")

Removed MSA from city names


In [10]:
df_users[["city", "state"]] = df_users.city.str.rsplit(", ", n=1, expand=True)
df_users["city"] = df_users["city"].str.lower()
df_users["state"] = df_users["state"].str.upper()
df_users.sample(5, random_state=42)

,age,city,reg_date,plan,churn_date,state
user_id,,,,,,
1361,45,birmingham-hoover,2018-04-14,surf,NaT,AL
1073,30,chicago-naperville-elgin,2018-04-06,surf,NaT,IL-IN-WI
1374,55,louisville/jefferson county,2018-02-14,surf,NaT,KY-IN
1155,19,miami-fort lauderdale-west palm beach,2018-02-21,ultimate,NaT,FL
1104,20,chicago-naperville-elgin,2018-12-23,ultimate,NaT,IL-IN-WI


In [11]:
# df_users = df_users.assign(month=df_users["reg_date"].dt.month, year=df_users["reg_date"].dt.year)

df_users["year_month"] = df_users["reg_date"].dt.to_period("M")
df_users = mdp.downcast_dtypes(df_users)

df_users.to_parquet("../data/interim/df_users.parquet", compression='brotli')

### Internet: Data Quality Check

In [12]:
url = data_url["internet"]
_ = pd.read_csv(url, nrows=2)
display(_)
_.info()

,id,user_id,session_date,mb_used
0,1000_13,1000,2018-12-29,89.86
1,1000_204,1000,2018-12-31,0.00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            2 non-null      object 
 1   user_id       2 non-null      int64  
 2   session_date  2 non-null      object 
 3   mb_used       2 non-null      float64
dtypes: float64(1), int64(1), object(2)
memory usage: 192.0+ bytes


In [13]:
df_internet = pd.read_csv(url,index_col='id', parse_dates=['session_date'])

mdp.check_data_quality(df_internet)

### Data Quality Report

#### Missing Values

<span style="color:green">No missing values found</span>

#### Duplicate Rows

<span style="color:orange">1731 duplicate rows found</span>

...

<span style="color:green">No highly correlated columns found</span>

#### Data Quality Report

This is a summary report. Change verbose to 1 to see more details on each DQ issue.
There are 1731 duplicate rows in your dataset
    Alert: Dropping duplicate rows can sometimes cause your column data types to change to object!
    All variables classified into correct types.


,DQ Issue
The Good News,"There are no duplicate columns in this datatset, There are no ID columns in the dataset., There are no zero-variance or low information columns in the dataset., There are no columns with missing values in the dataset, There are no categorical columns with rare categories (< 1 percent) in this dataset, There are no columns with infinite values in this dataset , There are no columns with mixed (more than one) dataypes in this dataset, There are no high cardinality columns in this dataset, There are no highly correlated columns in the dataset., There is no target given. Hence no target leakage columns detected in the dataset"
The Bad News,"There are 1731 duplicate rows in the dataset. De-Dup these rows using Fix_DQ., There are 1 date-time vars in the dataset. Make sure you transform them before modeling., There are 2 numerical columns, some with outliers. Remove them or use robust statistics."


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
user_id,int64,0.0,0,1000.0,1499.0,No issue
session_date,datetime64[ns],0.0,0,,,Possible date-time colum: transform before modeling step.
mb_used,float64,0.0,NA,0.0,1693.47,Column has 455 outliers greater than upper bound (1174.70) or lower than lower bound(-468.35). Cap them or remove them.


In [14]:
# Use assign to include a year and month columns
df_internet = df_internet.assign(year_month=df_internet['session_date'].dt.to_period('M'))
df_internet['gb_used'] = df_internet['mb_used'].div(1024)
df_internet = mdp.downcast_dtypes(df_internet.drop('mb_used', axis=1))

df_internet.to_parquet("../data/clean/internet.parquet", compression='brotli')

### SMS: Data Quality Check

In [15]:
url = data_url["sms"]
_ = pd.read_csv(url, nrows=2)
display(_)
_.info()

,id,user_id,message_date
0,1000_125,1000,2018-12-27
1,1000_160,1000,2018-12-31


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            2 non-null      object
 1   user_id       2 non-null      int64 
 2   message_date  2 non-null      object
dtypes: int64(1), object(2)
memory usage: 176.0+ bytes


In [16]:
df_sms = pd.read_csv(url,index_col='id', parse_dates=['message_date'])

mdp.check_data_quality(df_sms)

### Data Quality Report

#### Missing Values

<span style="color:green">No missing values found</span>

#### Duplicate Rows

<span style="color:orange">42808 duplicate rows found</span>

...

<span style="color:green">No highly correlated columns found</span>

#### Data Quality Report

This is a summary report. Change verbose to 1 to see more details on each DQ issue.
There are 42808 duplicate rows in your dataset
    Alert: Dropping duplicate rows can sometimes cause your column data types to change to object!
    All variables classified into correct types.


,DQ Issue
The Good News,"There are no duplicate columns in this datatset, There are no ID columns in the dataset., There are no zero-variance or low information columns in the dataset., There are no columns with missing values in the dataset, There are no categorical columns with rare categories (< 1 percent) in this dataset, There are no columns with infinite values in this dataset , There are no columns with mixed (more than one) dataypes in this dataset, There are no numeric columns with outliers in this dataset, There are no high cardinality columns in this dataset, There are no highly correlated columns in the dataset., There is no target given. Hence no target leakage columns detected in the dataset"
The Bad News,"There are 42808 duplicate rows in the dataset. De-Dup these rows using Fix_DQ., There are 1 date-time vars in the dataset. Make sure you transform them before modeling."


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
user_id,int64,0.0,1,1000.0,1497.0,No issue
message_date,datetime64[ns],0.0,1,,,Possible date-time colum: transform before modeling step.


In [17]:
# Use assign to include a year and month columns
df_sms = df_sms.assign(year_month=df_sms['message_date'].dt.to_period("M"))
df_sms = mdp.downcast_dtypes(df_sms)

df_sms.to_parquet("../data/clean/sms.parquet", compression='brotli')

### Calls: Data Quality Check

In [18]:
url = data_url["calls"]
_ = pd.read_csv(url, nrows=2)
display(_)
_.info()

,id,user_id,call_date,duration
0,1000_93,1000,2018-12-27,8.52
1,1000_145,1000,2018-12-27,13.66


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         2 non-null      object 
 1   user_id    2 non-null      int64  
 2   call_date  2 non-null      object 
 3   duration   2 non-null      float64
dtypes: float64(1), int64(1), object(2)
memory usage: 192.0+ bytes


In [19]:
df_calls = pd.read_csv(url,index_col='id', parse_dates=['call_date'])

mdp.check_data_quality(df_calls)

### Data Quality Report

#### Missing Values

<span style="color:green">No missing values found</span>

#### Duplicate Rows

<span style="color:orange">5882 duplicate rows found</span>

...

<span style="color:green">No highly correlated columns found</span>

#### Data Quality Report

This is a summary report. Change verbose to 1 to see more details on each DQ issue.
There are 5882 duplicate rows in your dataset
    Alert: Dropping duplicate rows can sometimes cause your column data types to change to object!
    All variables classified into correct types.


,DQ Issue
The Good News,"There are no duplicate columns in this datatset, There are no ID columns in the dataset., There are no zero-variance or low information columns in the dataset., There are no columns with missing values in the dataset, There are no categorical columns with rare categories (< 1 percent) in this dataset, There are no columns with infinite values in this dataset , There are no columns with mixed (more than one) dataypes in this dataset, There are no high cardinality columns in this dataset, There are no highly correlated columns in the dataset., There is no target given. Hence no target leakage columns detected in the dataset"
The Bad News,"There are 5882 duplicate rows in the dataset. De-Dup these rows using Fix_DQ., There are 1 date-time vars in the dataset. Make sure you transform them before modeling., There are 2 numerical columns, some with outliers. Remove them or use robust statistics."


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
user_id,int64,0.0,0,1000.0,1499.0,No issue
call_date,datetime64[ns],0.0,0,,,Possible date-time colum: transform before modeling step.
duration,float64,0.0,NA,0.0,37.6,Column has 604 outliers greater than upper bound (24.38) or lower than lower bound(-11.49). Cap them or remove them.


1. No hay datos ausentes en la tabla.
2. `duration` contiene los minutos en decimales, Megaline cobra el minuto completo (usar np.ceil) para resolver este problema.

In [20]:
df_calls['minutes'] = np.ceil(df_calls['duration'])  # redondeo hacia arriba

In [21]:
# Use assign to include a year and month columns
df_calls = df_calls.assign(year_month=df_calls['call_date'].dt.to_period("M"))
df_calls = mdp.downcast_dtypes(df_calls.drop('duration', axis=1))

df_calls.to_parquet("../data/clean/calls.parquet", compression='brotli')

### Planes: Data Quality Check

In [22]:
df_plans = pd.read_csv(data_url["plans"])
df_plans

,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name
0,50,15360,500,20,10,0.03,0.03,surf
1,1000,30720,3000,70,7,0.01,0.01,ultimate


`mb_per_month_included` debe convertirse a GB usando la formula (1gb = 1024 Mb)

In [23]:
#Megaline cobra por gb no mb, creo una col con los valores en Gb (1 GB = 1024 Mb)

df_plans["gb_per_month_included"] = df_plans["mb_per_month_included"]/1024
del df_plans["mb_per_month_included"]
df_plans = mdp.downcast_dtypes(df_plans)

In [24]:
df_plans = df_plans.rename(
    columns={
        # "plan_name": "plan",
        "usd_monthly_pay": "monthly_plan_fee",
        
        "usd_per_gb": "price_gb",
        "usd_per_message": "price_sms",
        "usd_per_minute": "price_minute",

        "gb_per_month_included": "included_gb",
        "messages_included": "included_sms",
        "minutes_included": "included_minutes",

    }
)
df_plans.to_parquet("../data/clean/plans.parquet", compression='brotli')

## Agregar datos por usuario

In [25]:
# Calcula el número de llamadas hechas por cada usuario al mes y los minutos totales
user_calls = df_calls.groupby(["user_id", "year_month"]).agg(
    calls_count=pd.NamedAgg(column='minutes', aggfunc='size'),
    minutes_sum=pd.NamedAgg(column='minutes', aggfunc='sum')
)

user_calls

calls_count  minutes_sum
user_id year_month                          
1000    2018-12         16          124.0   
1001    2018-08         27          182.0   
        2018-09         49          315.0   
        2018-10         65          393.0   
        2018-11         64          426.0   
...                         ...          ...
1498    2018-12         39          339.0   
1499    2018-09         41          346.0   
        2018-10         53          385.0   
        2018-11         45          308.0   
        2018-12         65          496.0   

[2258 rows x 2 columns]

In [26]:
# Calcula el número de mensajes enviados por cada usuario al mes. (as a DataFrame)
user_sms = df_sms.groupby(["user_id", "year_month"]).size().to_frame(name="sms_count")
user_sms

sms_count
user_id year_month           
1000    2018-12        11    
1001    2018-08        30    
        2018-09        44    
        2018-10        53    
        2018-11        36    
...                       ...
1496    2018-09        21    
        2018-10        18    
        2018-11        13    
        2018-12        11    
1497    2018-12        50    

[1806 rows x 1 columns]

In [27]:
# Calcula el volumen del tráfico de Internet usado por cada usuario al mes. (as a DataFrame)
user_internet = df_internet.groupby(["user_id", "year_month"]).agg(
    gb_used=pd.NamedAgg(column='gb_used', aggfunc='sum')
)


#ejemplo en instrucciones (web/mes 1205 mb -> 2gb)
user_internet['gb_used'] = user_internet['gb_used'].apply(np.ceil)  #round up por mes
user_internet

gb_used
user_id year_month         
1000    2018-12       2.0  
1001    2018-08       7.0  
        2018-09      14.0  
        2018-10      22.0  
        2018-11      19.0  
...                     ...
1498    2018-12      23.0  
1499    2018-09      13.0  
        2018-10      20.0  
        2018-11      17.0  
        2018-12      22.0  

[2277 rows x 1 columns]

In [28]:
# add year_month as second index for df_users
df_users = df_users.set_index('year_month', append=True)

In [29]:
user_monthly_data = (
    df_users[['plan', 'churn_date']].join(user_calls, how='outer')
            .join(user_sms, how='outer')
            .join(user_internet, how='outer')
    ).rename(columns=
         {
            "plan":"plan_name",
            "minutes_sum": "total_minutes",
            "sms_count": "total_sms",
            "gb_used":"total_gb"
          }
    ).sort_index(level=0)

In [30]:
user_monthly_data[["calls_count", "total_minutes", "total_sms", "total_gb"]] = user_monthly_data[["calls_count", "total_minutes", "total_sms", "total_gb"]].fillna(0)
user_monthly_data['plan_name'] = user_monthly_data.groupby(level=0)['plan_name'].fillna(method='ffill')

C:\Users\Guill\AppData\Local\Temp\ipykernel_19604\963937332.py:2: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  user_monthly_data['plan_name'] = user_monthly_data.groupby(level=0)['plan_name'].fillna(method='ffill')
C:\Users\Guill\AppData\Local\Temp\ipykernel_19604\963937332.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  user_monthly_data['plan_name'] = user_monthly_data.groupby(level=0)['plan_name'].fillna(method='ffill')


In [31]:
dq_report(user_monthly_data)

This is a summary report. Change verbose to 1 to see more details on each DQ issue.
There are 235 duplicate rows in your dataset
    Alert: Dropping duplicate rows can sometimes cause your column data types to change to object!
    All variables classified into correct types.


C:\Users\Guill\AppData\Local\Temp\ipykernel_19604\584651055.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dq_report(user_monthly_data)


,DQ Issue
The Good News,"There are no duplicate columns in this datatset, There are no ID columns in the dataset., There are no date-time vars in this dataset, There are no categorical columns with rare categories (< 1 percent) in this dataset, There are no columns with infinite values in this dataset , There are no columns with mixed (more than one) dataypes in this dataset, There are no high cardinality columns in this dataset, There is no target given. Hence no target leakage columns detected in the dataset"
The Bad News,"There are 235 duplicate rows in the dataset. De-Dup these rows using Fix_DQ., These are zero-variance or low information columns in the dataset. Remove them before modeling., There are 4 numerical columns, some with outliers. Remove them or use robust statistics., There are 1 columns with >= 0.8 correlation in the dataset. Drop one of them or use dimensionality reduction techniques."


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
plan_name,object,0.000000,0,,,No issue
churn_date,datetime64[ns],98.528775,1,,,"Possible Zero-variance or low information colum: drop before modeling step., 2277 missing values. Impute them with mean, median, mode, or a constant value such as 123."
calls_count,float64,0.000000,NA,0.0,205.0,Column has 43 outliers greater than upper bound (142.00) or lower than lower bound(-26.00). Cap them or remove them.
total_minutes,float32,0.000000,NA,0.0,1510.0,"Column has 42 outliers greater than upper bound (1035.50) or lower than lower bound(-204.50). Cap them or remove them., Column has a high correlation with ['calls_count']. Consider dropping one of them."
total_sms,float64,0.000000,NA,0.0,266.0,Column has 58 outliers greater than upper bound (123.00) or lower than lower bound(-69.00). Cap them or remove them.
total_gb,float32,0.000000,NA,0.0,70.0,Column has 59 outliers greater than upper bound (34.50) or lower than lower bound(-1.50). Cap them or remove them.


In [32]:
mdp.downcast_dtypes(user_monthly_data).to_parquet("../data/clean/user_monthly_data.parquet", compression='brotli')

# Full

In [33]:
# Añade la información de la tarifa
df = (user_monthly_data
        .reset_index()
        .merge(df_plans, on='plan_name', how='left')
        .set_index(["user_id", "year_month"])
)
df = df.assign(
    excess_usage=(
    (df['total_gb'] - df["included_gb"]).clip(0) * df['price_gb'] +
    (df['total_minutes'] - df["included_minutes"]).clip(0) * df['price_minute'] +
    (df['total_sms'] - df["included_sms"]).clip(0) * df['price_sms']
    ),
    total_payment=lambda x: x['monthly_plan_fee'] + x['excess_usage']
).drop("churn_date", axis=1)
df.to_parquet("../data/clean/usage_data.parquet", compression='brotli')

In [34]:
#join user data: usage_data
df = df[sorted(df.columns)]
(df.join(df_users)[["plan_name", "state", "monthly_plan_fee",
                    'calls_count', 'total_minutes', 'total_sms', 'total_gb',
                    "excess_usage", "total_payment"]]
 .to_parquet("../data/clean/usage_data.parquet", compression='brotli'))